In [120]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [121]:
df = pd.read_csv("./datasets/vehicles_2020-21/vehicles_2020.csv")
result = df


In [122]:
result

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,14,19694316,4322767,2020-06-06,18:30,fdc195a7-8127-4c00-834d-bac78b0cf88e,NaN,Bike,NaN,NaN,...,Going Straight Ahead,Center Front End,No Damage,No Damage,No Damage,No Damage,N,NaN,Traffic Control Disregarded,Unspecified
1,221,19693462,4322092,2020-06-22,7:30,eadf72ff-d03f-430d-8505-d9db881c3332,NY,Sedan,TOYT -CAR/SUV,NaN,...,Parked,Center Front End,Center Front End,Left Front Bumper,Right Front Bumper,No Damage,N,NaN,Unspecified,Unspecified
2,238,19693299,4322349,2020-06-22,18:10,7675ccbe-68b6-4842-8998-ebe005cc0d5d,NY,Van,TOYT-TRUCK/BUS,NaN,...,Going Straight Ahead,Right Side Doors,Right Side Doors,NaN,NaN,NaN,N,NaN,Other Vehicular,Other Vehicular
3,410,19693635,4322428,2020-06-22,15:37,63b1633c-fba4-4c6d-aff4-8dfea98db833,NaN,Sedan,NaN,NaN,...,Other*,Other,Other,Other,Other,Other,N,NaN,Unspecified,Unspecified
4,972,19693330,4322338,2020-06-22,10:19,97331e7e-4d04-40dc-8593-2c0dc0c5b032,NY,Station Wagon/Sport Utility Vehicle,MERZ -CAR/SUV,NaN,...,Parked,Left Front Bumper,Left Front Bumper,NaN,NaN,NaN,N,NaN,Driver Inattention/Distraction,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231162,3704288,19694062,4322520,2020-06-10,0:00,e1c0068a-2544-470d-832a-a664104498ad,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Unspecified,NaN,NaN,NaN
231163,3704289,19692445,4321864,2020-06-21,22:40,46666aa2-050a-4d0c-bc9a-418e812ac0d1,NJ,Sedan,NaN,NaN,...,Making Right Turn,Left Rear Bumper,Left Rear Bumper,Left Rear Quarter Panel,NaN,NaN,Unspecified,NaN,Unspecified,Unspecified
231164,3704290,19693956,4322491,2020-06-17,21:20,f3347374-12f6-4c1f-a3cd-6c0212d7d4ca,NY,Sedan,NISS -CAR/SUV,NaN,...,Making Left Turn,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,Center Front End,NaN,Unspecified,NaN,Unspecified,Unspecified
231165,3704291,19693403,4322315,2020-06-21,6:50,cf74942e-9343-4e75-bd42-9cca4b542194,NY,Station Wagon/Sport Utility Vehicle,CHEV -CAR/SUV,NaN,...,Parked,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,No Damage,No Damage,N,NaN,Unspecified,Unspecified


In [123]:
result.isna().sum()

Unnamed: 0                          0
UNIQUE_ID                           0
COLLISION_ID                        0
CRASH_DATE                          0
CRASH_TIME                          0
VEHICLE_ID                          0
STATE_REGISTRATION              35052
VEHICLE_TYPE                    26212
VEHICLE_MAKE                    39676
VEHICLE_MODEL                  231167
VEHICLE_YEAR                    41236
TRAVEL_DIRECTION                16055
VEHICLE_OCCUPANTS               28486
DRIVER_SEX                      75659
DRIVER_LICENSE_STATUS           83003
DRIVER_LICENSE_JURISDICTION     83848
PRE_CRASH                       19159
POINT_OF_IMPACT                 19513
VEHICLE_DAMAGE                  22064
VEHICLE_DAMAGE_1                90652
VEHICLE_DAMAGE_2               124028
VEHICLE_DAMAGE_3               150691
PUBLIC_PROPERTY_DAMAGE              0
PUBLIC_PROPERTY_DAMAGE_TYPE    228124
CONTRIBUTING_FACTOR_1           15268
CONTRIBUTING_FACTOR_2           17776
dtype: int64

In [124]:
result = result.dropna(subset=['VEHICLE_TYPE'])
result

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
0,14,19694316,4322767,2020-06-06,18:30,fdc195a7-8127-4c00-834d-bac78b0cf88e,NaN,Bike,NaN,NaN,...,Going Straight Ahead,Center Front End,No Damage,No Damage,No Damage,No Damage,N,NaN,Traffic Control Disregarded,Unspecified
1,221,19693462,4322092,2020-06-22,7:30,eadf72ff-d03f-430d-8505-d9db881c3332,NY,Sedan,TOYT -CAR/SUV,NaN,...,Parked,Center Front End,Center Front End,Left Front Bumper,Right Front Bumper,No Damage,N,NaN,Unspecified,Unspecified
2,238,19693299,4322349,2020-06-22,18:10,7675ccbe-68b6-4842-8998-ebe005cc0d5d,NY,Van,TOYT-TRUCK/BUS,NaN,...,Going Straight Ahead,Right Side Doors,Right Side Doors,NaN,NaN,NaN,N,NaN,Other Vehicular,Other Vehicular
3,410,19693635,4322428,2020-06-22,15:37,63b1633c-fba4-4c6d-aff4-8dfea98db833,NaN,Sedan,NaN,NaN,...,Other*,Other,Other,Other,Other,Other,N,NaN,Unspecified,Unspecified
4,972,19693330,4322338,2020-06-22,10:19,97331e7e-4d04-40dc-8593-2c0dc0c5b032,NY,Station Wagon/Sport Utility Vehicle,MERZ -CAR/SUV,NaN,...,Parked,Left Front Bumper,Left Front Bumper,NaN,NaN,NaN,N,NaN,Driver Inattention/Distraction,Unspecified
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231161,3704287,19693897,4322405,2020-06-20,14:30,f5c1ef6e-5557-475b-9169-e5b1898d8a4e,NY,Sedan,INFI -CAR/SUV,NaN,...,Going Straight Ahead,Right Front Bumper,Other,Right Front Bumper,Right Front Bumper,Right Front Bumper,N,NaN,Unspecified,Unspecified
231163,3704289,19692445,4321864,2020-06-21,22:40,46666aa2-050a-4d0c-bc9a-418e812ac0d1,NJ,Sedan,NaN,NaN,...,Making Right Turn,Left Rear Bumper,Left Rear Bumper,Left Rear Quarter Panel,NaN,NaN,Unspecified,NaN,Unspecified,Unspecified
231164,3704290,19693956,4322491,2020-06-17,21:20,f3347374-12f6-4c1f-a3cd-6c0212d7d4ca,NY,Sedan,NISS -CAR/SUV,NaN,...,Making Left Turn,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,Center Front End,NaN,Unspecified,NaN,Unspecified,Unspecified
231165,3704291,19693403,4322315,2020-06-21,6:50,cf74942e-9343-4e75-bd42-9cca4b542194,NY,Station Wagon/Sport Utility Vehicle,CHEV -CAR/SUV,NaN,...,Parked,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,No Damage,No Damage,N,NaN,Unspecified,Unspecified


In [125]:
result_grped = result.groupby('VEHICLE_TYPE').count()
result_grped

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_MAKE,VEHICLE_MODEL,VEHICLE_YEAR,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
VEHICLE_TYPE,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,0,1,1
18 WEELER,1,1,1,1,1,1,1,1,0,0,...,1,1,1,1,0,0,1,0,1,1
18 WHEELER,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,0,1,0,1,1
1C,1,1,1,1,1,1,1,1,0,1,...,1,1,1,1,1,1,1,0,1,1
2 dr sedan,22,22,22,22,22,22,22,20,0,22,...,22,22,22,16,8,6,22,0,22,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
util,1,1,1,1,1,1,1,0,0,1,...,1,1,1,1,1,1,1,0,1,1
utility,3,3,3,3,3,3,3,2,0,3,...,3,3,3,3,2,2,3,0,3,3
van,64,64,64,64,64,64,61,52,0,55,...,63,64,62,42,35,32,64,0,64,63


In [126]:
result["CRASH_DATE"] = pd.to_datetime(result["CRASH_DATE"])
result["ACCIDENT_MONTH"] = result["CRASH_DATE"].dt.month
result

/tmp/ipykernel_6653/3941619500.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["CRASH_DATE"] = pd.to_datetime(result["CRASH_DATE"])
/tmp/ipykernel_6653/3941619500.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result["ACCIDENT_MONTH"] = result["CRASH_DATE"].dt.month


,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2,ACCIDENT_MONTH
0,14,19694316,4322767,2020-06-06,18:30,fdc195a7-8127-4c00-834d-bac78b0cf88e,NaN,Bike,NaN,NaN,...,Center Front End,No Damage,No Damage,No Damage,No Damage,N,NaN,Traffic Control Disregarded,Unspecified,6
1,221,19693462,4322092,2020-06-22,7:30,eadf72ff-d03f-430d-8505-d9db881c3332,NY,Sedan,TOYT -CAR/SUV,NaN,...,Center Front End,Center Front End,Left Front Bumper,Right Front Bumper,No Damage,N,NaN,Unspecified,Unspecified,6
2,238,19693299,4322349,2020-06-22,18:10,7675ccbe-68b6-4842-8998-ebe005cc0d5d,NY,Van,TOYT-TRUCK/BUS,NaN,...,Right Side Doors,Right Side Doors,NaN,NaN,NaN,N,NaN,Other Vehicular,Other Vehicular,6
3,410,19693635,4322428,2020-06-22,15:37,63b1633c-fba4-4c6d-aff4-8dfea98db833,NaN,Sedan,NaN,NaN,...,Other,Other,Other,Other,Other,N,NaN,Unspecified,Unspecified,6
4,972,19693330,4322338,2020-06-22,10:19,97331e7e-4d04-40dc-8593-2c0dc0c5b032,NY,Station Wagon/Sport Utility Vehicle,MERZ -CAR/SUV,NaN,...,Left Front Bumper,Left Front Bumper,NaN,NaN,NaN,N,NaN,Driver Inattention/Distraction,Unspecified,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231161,3704287,19693897,4322405,2020-06-20,14:30,f5c1ef6e-5557-475b-9169-e5b1898d8a4e,NY,Sedan,INFI -CAR/SUV,NaN,...,Right Front Bumper,Other,Right Front Bumper,Right Front Bumper,Right Front Bumper,N,NaN,Unspecified,Unspecified,6
231163,3704289,19692445,4321864,2020-06-21,22:40,46666aa2-050a-4d0c-bc9a-418e812ac0d1,NJ,Sedan,NaN,NaN,...,Left Rear Bumper,Left Rear Bumper,Left Rear Quarter Panel,NaN,NaN,Unspecified,NaN,Unspecified,Unspecified,6
231164,3704290,19693956,4322491,2020-06-17,21:20,f3347374-12f6-4c1f-a3cd-6c0212d7d4ca,NY,Sedan,NISS -CAR/SUV,NaN,...,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,Center Front End,NaN,Unspecified,NaN,Unspecified,Unspecified,6
231165,3704291,19693403,4322315,2020-06-21,6:50,cf74942e-9343-4e75-bd42-9cca4b542194,NY,Station Wagon/Sport Utility Vehicle,CHEV -CAR/SUV,NaN,...,Right Front Bumper,Right Front Bumper,Right Front Quarter Panel,No Damage,No Damage,N,NaN,Unspecified,Unspecified,6


In [127]:
result_grped = result.groupby('ACCIDENT_MONTH').count()
result_grped

,Unnamed: 0,UNIQUE_ID,COLLISION_ID,CRASH_DATE,CRASH_TIME,VEHICLE_ID,STATE_REGISTRATION,VEHICLE_TYPE,VEHICLE_MAKE,VEHICLE_MODEL,...,PRE_CRASH,POINT_OF_IMPACT,VEHICLE_DAMAGE,VEHICLE_DAMAGE_1,VEHICLE_DAMAGE_2,VEHICLE_DAMAGE_3,PUBLIC_PROPERTY_DAMAGE,PUBLIC_PROPERTY_DAMAGE_TYPE,CONTRIBUTING_FACTOR_1,CONTRIBUTING_FACTOR_2
ACCIDENT_MONTH,,,,,,,,,,,,,,,,,,,,,
1,26477,26477,26477,26477,26477,26477,25842,26477,25578,0,...,26282,26278,26174,16949,12660,9570,26477,0,26382,26302
2,25406,25406,25406,25406,25406,25406,24803,25406,24573,0,...,25199,25214,25118,16325,12193,9205,25406,0,25313,25256
3,20480,20480,20480,20480,20480,20480,19865,20480,19666,0,...,20305,20320,20243,13253,9924,7466,20480,0,20388,20338
4,7395,7395,7395,7395,7395,7395,7049,7395,6961,0,...,7309,7324,7280,5227,4025,2921,7395,0,7351,7326
5,11227,11227,11227,11227,11227,11227,10492,11227,10124,0,...,11085,11105,11023,7691,5927,4393,11227,0,11156,11101
6,13904,13904,13904,13904,13904,13904,12799,13904,12417,0,...,13725,13738,13630,9521,7285,5370,13904,0,13816,13747
7,16668,16668,16668,16668,16668,16668,15339,16668,14893,0,...,16458,16472,16340,11468,8840,6643,16668,0,16570,16480
8,17611,17611,17611,17611,17611,17611,16155,17611,15760,0,...,17356,17377,17261,12153,9375,6941,17611,0,17505,17403
9,17353,17353,17353,17353,17353,17353,15834,17353,15467,0,...,17108,17097,16969,11645,8918,6632,17353,0,17246,17145
